# Target

- Solve a classification problem with deep neural networks (DNNs).
- Understand recursive neural networks (RNNs).

# Library

In [146]:
import random
import os
import argparse
from tqdm import tqdm

import numpy as np

from torch.utils.data import Dataset
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
import gc

# Helper function

- `random.seed(seed)`: Sets the seed for **Python's built-in** random module.
- `np.random.seed(seed)`: Sets the seed for **NumPy's** random number generator.
- `torch.manual_seed(seed)`: Sets the seed for **PyTorch on the CPU**.
- `torch.cuda.manual_seed(seed)`: Sets the seed for the **current GPU**.
- `torch.cuda.manual_seed_all(seed)`: Sets the seed for **all GPUs**.
- `torch.backends.cudnn.benchmark = False`: Disables the CuDNN benchmark mode.
- `torch.backends.cudnn.deterministic = True`: Forces CuDNN to use deterministic algorithms, which helps in reproducibility.

`torch.backends.cudnn.benchmark` 是 PyTorch 的一個設置，用來啟用或禁用 CuDNN 的基準模式：

- True：讓 CuDNN 尋找最佳的卷積算法，可能提高訓練速度，但結果可能不可重現。
- False：使用固定的算法，確保結果可重現，通常用於測試和比較實驗結果。

**在追求可重現性時，可設置為 False**

In [149]:
def same_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmarkbe = False
    torch.backends.cudnn.deterministic = True

- 函數：

`load_feat(path)`：

    從指定路徑加載 PyTorch 張量。

`shift(x, n)`：

    將張量 x 向左或向右移動 n 個位置。
    根據移動方向用第一或最後一個元素填充。

`concat_feat(x, concat_n)`：

    在每個幀周圍串聯 concat_n 幀。
    確保 concat_n 是奇數。
    使用 shift 函數來正確調整每個幀。

- 主函數：
`preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8)`：

    用於訓練、驗證或測試的數據預處理。
    加載和串聯特徵。
    根據 train_ratio 將數據拆分為訓練集和驗證集。
    
    - 步驟：

        - 加載數據：

            讀取標籤並拆分數據集。
            使用 load_feat 加載特徵文件。
            特徵串聯：

            使用 concat_feat 串聯幀以提供上下文。
            數據準備：

            為特徵 (X) 和標籤 (y) 準備張量。

        - 輸出：

            返回處理好的訓練/驗證集的特徵和標籤張量，或僅返回測試集的特徵。
            關鍵點：
            concat_nframes 必須是奇數以確保對稱上下文。
            train_ratio 控制訓練和驗證之間的拆分比例。
            預定義的最大長度 (max_len) 保證了內存的高效分配，可以根據需要調整。
            使用 tqdm 來追踪進度。

---

In [140]:
def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x: torch.Tensor, n: int) -> torch.Tensor:
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]
    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x: torch.Tensor, concat_n: int):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n) 
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)


**Explain**

In [99]:
test = torch.Tensor([[1,2,3],
                     [4,5,6],
                     [7,8,9],
                     [10,11,12]])

shift(test,2)

tensor([[ 7.,  8.,  9.],
        [10., 11., 12.],
        [10., 11., 12.],
        [10., 11., 12.]])

In [100]:
test.shape

torch.Size([4, 3])

In [101]:
test.repeat(1, 3).shape

torch.Size([4, 9])

In [102]:
seq_len, feature_len = test.size(0), test.size(1) # 4*3
test = test.repeat(1, 3) # 4*9
test.view(seq_len, 3, feature_len) # 4 *3 *3

tensor([[[ 1.,  2.,  3.],
         [ 1.,  2.,  3.],
         [ 1.,  2.,  3.]],

        [[ 4.,  5.,  6.],
         [ 4.,  5.,  6.],
         [ 4.,  5.,  6.]],

        [[ 7.,  8.,  9.],
         [ 7.,  8.,  9.],
         [ 7.,  8.,  9.]],

        [[10., 11., 12.],
         [10., 11., 12.],
         [10., 11., 12.]]])

In [105]:
test = test.view(seq_len, 3, feature_len).permute(1, 0, 2)

In [106]:
test  # 3*4*3 permute() rearange tensor

tensor([[[ 1.,  2.,  3.],
         [ 4.,  5.,  6.],
         [ 7.,  8.,  9.],
         [10., 11., 12.]],

        [[ 1.,  2.,  3.],
         [ 4.,  5.,  6.],
         [ 7.,  8.,  9.],
         [10., 11., 12.]],

        [[ 1.,  2.,  3.],
         [ 4.,  5.,  6.],
         [ 7.,  8.,  9.],
         [10., 11., 12.]]])

In [107]:
mid = (3 // 2) # 1
for r_idx in range(1, mid+1): # 1
    a = shift(test[mid + r_idx], r_idx)
    test[mid + r_idx, :] = a
    print(a)

    b = shift(test[mid - r_idx], -r_idx)
    test[mid - r_idx, :] = b
    print(b)

tensor([[ 4.,  5.,  6.],
        [ 7.,  8.,  9.],
        [10., 11., 12.],
        [10., 11., 12.]])
tensor([[1., 2., 3.],
        [1., 2., 3.],
        [4., 5., 6.],
        [7., 8., 9.]])


In [109]:
test.permute(1, 0, 2).view(seq_len, 3 * feature_len) # 4*9

tensor([[ 1.,  2.,  3.,  1.,  2.,  3.,  4.,  5.,  6.],
        [ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9.],
        [ 4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.],
        [ 7.,  8.,  9., 10., 11., 12., 10., 11., 12.]])

---

In [141]:
def preprocess_data(split: str, feat_dir, phone_path, concat_nframes, train_ratio=0.8):
    class_num = 41 # NOTE: pre-computed, should not need change

    if split == 'train' or split == 'val':
        mode = 'train'
    elif split == 'test':
        mode = 'test'
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    label_dict = {}
    if mode == 'train':
        for line in open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines():
            line = line.strip('\n').split(' ')
            label_dict[line[0]] = [int(p) for p in line[1:]]
        
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.shuffle(usage_list)
        train_len = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:train_len] if split == 'train' else usage_list[train_len:]

    elif mode == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)
    if mode == 'train':
        y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode == 'train':
            label = torch.LongTensor(label_dict[fname])

        X[idx: idx + cur_len, :] = feat
        if mode == 'train':
            y[idx: idx + cur_len] = label

        idx += cur_len

    X = X[:idx, :]
    if mode == 'train':
        y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode == 'train':
        print(y.shape)
        return X, y
    else:
        return X

**Explain**

In [131]:
label_dict = {}
for line in open("/kaggle/input/libraphone/libriphone/train_labels.txt").readlines():
    print(len(line))
    line = line.strip('\n').split(' ')
    print(len(line))
    label_dict[line[0]] = [int(p) for p in line[1:]]
    print(label_dict)
    break



2121
841
{'4830-25898-0031': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12, 12, 12, 12, 12, 12, 12, 27, 27, 27, 38, 38, 38, 38, 38, 38, 38, 35, 35, 35, 35, 35, 25, 25, 25, 25, 25, 25, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 32, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 37, 37, 37, 37, 37, 30, 30, 30, 30, 30, 30, 30, 30, 27, 27, 27, 4, 4, 4, 4, 31, 31, 31, 31, 31, 30, 30, 30, 30, 30, 30, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 30, 30, 30, 30, 30, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 8, 8, 8, 8, 8, 8, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 27, 27, 27, 27, 27, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 27, 27, 27, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 37, 37, 37, 37, 37, 37, 14, 14, 14, 14, 14, 14, 14, 4, 4, 4, 4, 4, 4, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 32, 32, 32, 32, 32, 32, 32, 

In [133]:
split = "train"

usage_list = open("/kaggle/input/libraphone/libriphone/train_split.txt").readlines()
print(f"read first: {usage_list[0]}")
random.shuffle(usage_list)
print(f"After shuffling: {usage_list[0]}")
train_len = int(len(usage_list) * 0.8)
usage_list = usage_list[:train_len] if split == 'train' else usage_list[train_len:]
print(f"After train split: {usage_list[0]}")
usage_list = [line.strip('\n') for line in usage_list]
print(f"Strip \\n: {usage_list[0]}")
print('[Dataset] - # phone classes: ' + str(41) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

read first: 4830-25898-0031

After shuffling: 2989-138028-0022

After train split: 2989-138028-0022

Strip \n: 2989-138028-0022
[Dataset] - # phone classes: 41, number of utterances for train: 2743


----

# Dataset


In [142]:
class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)

# Model

In [143]:
class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()

        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.block(x)
        return x


class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
        super().__init__()

        self.fc = nn.Sequential(
            BasicBlock(input_dim, hidden_dim),
            *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x = self.fc(x)
        return x

# Hyper-parameters

In [ ]:
parser = argparse.ArgumentParser(description='設定資料與訓練參數')

# 資料參數
parser.add_argument('--concat_nframes', type=int, default=3,
                    help='連接的幀數，n 必須為奇數 (總共 2k+1 = n 幀)')
parser.add_argument('--train_ratio', type=float, default=0.75,
                    help='訓練資料比例，其餘將用於驗證')

# 訓練參數
parser.add_argument('--seed', type=int, default=1213,
                    help='隨機種子')
parser.add_argument('--batch_size', type=int, default=512,
                    help='批次大小')
parser.add_argument('--num_epoch', type=int, default=10,
                    help='訓練的 epoch 數量')
parser.add_argument('--learning_rate', type=float, default=1e-4,
                    help='學習率')
parser.add_argument('--model_path', type=str, default='./model.ckpt',
                    help='模型儲存的路徑')

# 模型參數
parser.add_argument('--input_dim', type=int, default=39 * 3,
                    help='模型的輸入維度，不應更改此值')
parser.add_argument('--hidden_layers', type=int, default=2,
                    help='隱藏層的層數')
parser.add_argument('--hidden_dim', type=int, default=64,
                    help='隱藏層的維度')

args = parser.parse_args()

# 打印參數設定
print('資料參數:')
print(f'- 連接的幀數: {args.concat_nframes}')
print(f'- 訓練資料比例: {args.train_ratio}')

print('\n訓練參數:')
print(f'- 隨機種子: {args.seed}')
print(f'- 批次大小: {args.batch_size}')
print(f'- 訓練 epoch 數量: {args.num_epoch}')
print(f'- 學習率: {args.learning_rate}')
print(f'- 模型儲存路徑: {args.model_path}')

print('\n模型參數:')
print(f'- 輸入維度: {args.input_dim}')
print(f'- 隱藏層數量: {args.hidden_layers}')
print(f'- 隱藏層維度: {args.hidden_dim}')

In [144]:
# data prarameters
concat_nframes = 3              # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.75               # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 1213                        # random seed
batch_size = 512                # batch size
num_epoch = 10                   # the number of training epoch
learning_rate = 1e-4         # learning rate
model_path = './model.ckpt'     # the path where the checkpoint will be saved

# model parameters
input_dim = 39 * concat_nframes # the input dim of the model, you should not change the value
hidden_layers = 2               # the number of hidden layers
hidden_dim = 64                # the hidden dim

# Dataloader

In [150]:
same_seeds(seed)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

DEVICE: cpu


In [152]:
# preprocess data
train_X, train_y = preprocess_data(split='train',
                                   feat_dir='/kaggle/input/libraphone/libriphone/feat',
                                   phone_path='/kaggle/input/libraphone/libriphone',
                                   concat_nframes=concat_nframes,
                                   train_ratio=train_ratio)

val_X, val_y = preprocess_data(split='val',
                               feat_dir='/kaggle/input/libraphone/libriphone/feat',
                               phone_path='/kaggle/input/libraphone/libriphone',
                               concat_nframes=concat_nframes,
                               train_ratio=train_ratio)

[Dataset] - # phone classes: 41, number of utterances for train: 2571


2571it [00:18, 136.19it/s]


[INFO] train set
torch.Size([1588590, 117])
torch.Size([1588590])
[Dataset] - # phone classes: 41, number of utterances for val: 858


858it [00:02, 347.73it/s]

[INFO] val set
torch.Size([525078, 117])
torch.Size([525078])


In [153]:
# get dataset
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

In [154]:
# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

3780

In [164]:
# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

# Training

In [162]:
next(iter(train_loader))[0].shape

torch.Size([512, 117])

In [161]:
next(iter(train_loader))[1].shape

torch.Size([512])

In [168]:
# create model, define a loss function, and optimizer
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0
    
    # training
    model.train() # set the model to training mode
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels = batch
        features = features.to(device) # 512 * 117
        labels = labels.to(device)
        
        optimizer.zero_grad() 
        outputs = model(features)  # 512 * 41
        
        loss = criterion(outputs, labels)
        loss.backward() 
        optimizer.step() 
        
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        train_acc += (train_pred.detach() == labels.detach()).sum().item() # requires_grad as False
        train_loss += loss.item()
    
    # validation
    model.eval() # set the model to evaluation mode
    with torch.no_grad():
        for i, batch in enumerate(tqdm(val_loader)):
            features, labels = batch
            features = features.to(device) 
            labels = labels.to(device)
            outputs = model(features)
            
            loss = criterion(outputs, labels) 
            
            _, val_pred = torch.max(outputs, 1) 
            val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
            val_loss += loss.item()

    print(f'[{epoch+1:03d}/{num_epoch:03d}] Train Acc: {train_acc/len(train_set):3.5f} Loss: {train_loss/len(train_loader):3.5f} | Val Acc: {val_acc/len(val_set):3.5f} loss: {val_loss/len(val_loader):3.5f}')

    # if the model improves, save a checkpoint at this epoch
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), model_path)
        print(f'saving model with acc {best_acc/len(val_set):.5f}')

  1%|          | 17/3103 [00:00<00:40, 75.63it/s]

torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size

  2%|▏         | 52/3103 [00:00<00:22, 135.14it/s]

torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size

  3%|▎         | 90/3103 [00:00<00:18, 162.04it/s]

...

  5%|▌         | 163/3103 [00:01<00:16, 175.46it/s]

torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size

  6%|▌         | 193/3103 [00:01<00:19, 151.79it/s]


torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])
torch.Size([512, 117])
torch.Size([512, 41])


KeyboardInterrupt: 